In [228]:
import pandas as pd
from datetime import datetime
from pytz import timezone
import pytz
import glob
import os
import re

In [229]:
game_list_file = r'C:\Users\benny\OneDrive\Machine Learning\ML1030\Final Project\Final Project\games.csv'
NFL_teams_file = r'C:\Users\benny\OneDrive\Machine Learning\ML1030\Final Project\Final Project\NFLTeamAbbreviations.xlsx'
nfl_game_schedule = pd.read_csv(game_list_file)
print(nfl_game_schedule.head())
nfl_team_names = pd.read_excel(NFL_teams_file)
print(nfl_team_names.head())

   Week  Day    Date    Time            Winner/tie   At            Loser/tie  \
0     1  Thu  06-Sep  8:20PM   Philadelphia Eagles  NaN      Atlanta Falcons   
1     1  Sun  09-Sep  1:00PM  Tampa Bay Buccaneers    @   New Orleans Saints   
2     1  Sun  09-Sep  1:00PM     Minnesota Vikings  NaN  San Francisco 49ers   
3     1  Sun  09-Sep  1:00PM        Miami Dolphins  NaN     Tennessee Titans   
4     1  Sun  09-Sep  1:00PM  Jacksonville Jaguars    @      New York Giants   

   boxscore  PtsW  PtsL  YdsW  TOW  YdsL  TOL  
0  boxscore    18    12   232    2   299    1  
1  boxscore    48    40   529    0   475    2  
2  boxscore    24    16   343    1   327    4  
3  boxscore    27    20   342    2   336    3  
4  boxscore    20    15   305    1   324    2  
  Acronym          Franchise
0     ARI  Arizona Cardinals
1     ATL    Atlanta Falcons
2     BAL   Baltimore Ravens
3     BUF      Buffalo Bills
4     CAR  Carolina Panthers


In [230]:
eastern = pytz.timezone('US/Eastern')
nfl_game_schedule['Home'] = ""
nfl_game_schedule['Home_abbr'] = ""
nfl_game_schedule['Away'] = ""
nfl_game_schedule['Away_abbr'] = ""
nfl_game_schedule['Game'] = ""
nfl_game_schedule['Game Time'] = ""
for i in nfl_game_schedule.index:
    if nfl_game_schedule.loc[i,'At'] != "@":
        nfl_game_schedule.loc[i,'Home'] = nfl_game_schedule.loc[i,'Winner/tie']
        nfl_game_schedule.loc[i,'Away'] = nfl_game_schedule.loc[i,'Loser/tie']
    else:
        nfl_game_schedule.loc[i,'Away'] = nfl_game_schedule.loc[i,'Winner/tie']
        nfl_game_schedule.loc[i,'Home'] = nfl_game_schedule.loc[i,'Loser/tie']
    df_home = nfl_team_names[nfl_team_names['Franchise'] == nfl_game_schedule.loc[i,'Home']]['Acronym']
    df_away = nfl_team_names[nfl_team_names['Franchise'] == nfl_game_schedule.loc[i,'Away']]['Acronym']
    nfl_game_schedule.loc[i,'Home_abbr'] = df_home.values
    nfl_game_schedule.loc[i,'Away_abbr'] = df_away.values
    nfl_game_schedule.loc[i,'Game'] = df_away.values + "@" + df_home.values
    timestamp_str = nfl_game_schedule.loc[i,'Date'] + "-2018" + " " + nfl_game_schedule.loc[i,'Time']
    nfl_game_schedule.loc[i,'Game Time'] = eastern.localize(datetime.strptime(timestamp_str, '%d-%b-%Y %I:%M%p'),is_dst=True) 
    
nfl_game_schedule['Game Time'] = pd.DatetimeIndex(nfl_game_schedule['Game Time']).tz_convert(eastern)
print(nfl_game_schedule.head())    
nfl_game_schedule.to_csv(export_path+"\\games_data.csv")

   Week  Day    Date    Time            Winner/tie   At            Loser/tie  \
0     1  Thu  06-Sep  8:20PM   Philadelphia Eagles  NaN      Atlanta Falcons   
1     1  Sun  09-Sep  1:00PM  Tampa Bay Buccaneers    @   New Orleans Saints   
2     1  Sun  09-Sep  1:00PM     Minnesota Vikings  NaN  San Francisco 49ers   
3     1  Sun  09-Sep  1:00PM        Miami Dolphins  NaN     Tennessee Titans   
4     1  Sun  09-Sep  1:00PM  Jacksonville Jaguars    @      New York Giants   

   boxscore  PtsW  PtsL  YdsW  TOW  YdsL  TOL                 Home Home_abbr  \
0  boxscore    18    12   232    2   299    1  Philadelphia Eagles       PHI   
1  boxscore    48    40   529    0   475    2   New Orleans Saints        NO   
2  boxscore    24    16   343    1   327    4    Minnesota Vikings       MIN   
3  boxscore    27    20   342    2   336    3       Miami Dolphins       MIA   
4  boxscore    20    15   305    1   324    2      New York Giants       NYG   

                   Away Away_abbr     

In [231]:
path = r'C:\Users\benny\OneDrive\Machine Learning\ML1030\Final Project\Final Project\Data' 
all_csv_files = glob.glob(os.path.join(path, "*.csv"))
all_xlsx_files = glob.glob(os.path.join(path, "*.xlsx"))

print(len(all_csv_files))
print(len(all_xlsx_files))


75
101


In [232]:
li_csv = pd.DataFrame()
i = 1

for filename in all_csv_files:
    (head, tail) = os.path.split(filename)
    teams = re.search('NFL-(.+?)-data', tail).group(1)
    #print(tail)
    #print(teams)
    away=teams.split('@')[0]
    home=teams.split('@')[1]
    #print(away)
    #print(home)
    csv_df = pd.read_csv(filename)
    csv_df['Away'] = away
    csv_df['Home'] = home
    csv_df['Game'] = teams
    csv_df['Game Number'] = i
    i = i+1
    li_csv = pd.concat([li_csv,csv_df],sort=None)

print(li_csv.shape)    
    
li_xlsx = pd.DataFrame()

for filename in all_xlsx_files:
    (head, tail) = os.path.split(filename)
    teams = re.search('NFL-(.+?)-data', tail).group(1)
    #print(tail)
    #print(teams)
    away=teams.split('@')[0]
    home=teams.split('@')[1]
    #print(away)
    #print(home)
    xlsx_df = pd.read_excel(filename)
    xlsx_df['Away'] = away
    xlsx_df['Home'] = home
    xlsx_df['Game'] = teams
    xlsx_df['Game Number'] = i
    i = i+1
    li_xlsx = pd.concat([li_xlsx,xlsx_df],sort=None)

print(li_xlsx.shape)

sportsbook_data = pd.concat([li_csv,li_xlsx], sort=None, ignore_index=True)

print(sportsbook_data)



(289758, 15)
(128950, 15)
                                           Date       Book Away Moneyline  \
0       Sun Sep 02 2018 15:37:47 GMT+0000 (UTC)   AlfaBook            115   
1       Thu Aug 30 2018 20:54:20 GMT+0000 (UTC)  BravoBook            NaN   
2       Thu Aug 30 2018 21:54:19 GMT+0000 (UTC)  BravoBook            NaN   
3       Thu Aug 30 2018 22:54:20 GMT+0000 (UTC)  BravoBook            NaN   
4       Thu Aug 30 2018 23:54:20 GMT+0000 (UTC)  BravoBook            NaN   
5       Fri Aug 31 2018 00:54:20 GMT+0000 (UTC)  BravoBook            NaN   
6       Fri Aug 31 2018 13:54:19 GMT+0000 (UTC)  BravoBook            NaN   
7       Fri Aug 31 2018 14:54:19 GMT+0000 (UTC)  BravoBook            NaN   
8       Fri Aug 31 2018 15:54:20 GMT+0000 (UTC)  BravoBook            NaN   
9       Fri Aug 31 2018 16:54:21 GMT+0000 (UTC)  BravoBook            NaN   
10      Fri Aug 31 2018 17:54:20 GMT+0000 (UTC)  BravoBook            NaN   
11      Fri Aug 31 2018 18:54:20 GMT+0000 (UTC)  B

In [233]:
sportsbook_data.head()

,Date,Book,Away Moneyline,Home Moneyline,Away Pointspread points,Away Pointspread odds,Home Pointspread points,Home Pointspread odds,Total points,Over odds,Under odds,Away,Home,Game,Game Number
0,Sun Sep 02 2018 15:37:47 GMT+0000 (UTC),AlfaBook,115,-135,2.5,-110.0,-2.5,-110.0,45.5,-108.0,-112.0,ATL,PHI,ATL@PHI,1
1,Thu Aug 30 2018 20:54:20 GMT+0000 (UTC),BravoBook,NaN,NaN,3,-119.0,-3,-101.0,45.5,-105.0,-115.0,ATL,PHI,ATL@PHI,1
2,Thu Aug 30 2018 21:54:19 GMT+0000 (UTC),BravoBook,NaN,NaN,3,-110.0,-3,-110.0,45.5,-105.0,-115.0,ATL,PHI,ATL@PHI,1
3,Thu Aug 30 2018 22:54:20 GMT+0000 (UTC),BravoBook,NaN,NaN,3,-110.0,-3,-110.0,45.5,-105.0,-115.0,ATL,PHI,ATL@PHI,1
4,Thu Aug 30 2018 23:54:20 GMT+0000 (UTC),BravoBook,NaN,NaN,3,-110.0,-3,-110.0,45.5,-105.0,-115.0,ATL,PHI,ATL@PHI,1


In [234]:
sportsbook_data['timestamp'] =  pd.to_datetime(sportsbook_data.Date.str.slice(0,24), format='%a %b %d %Y %H:%M:%S')
#eastern = pytz.timezone('US/Eastern')
sportsbook_data['timestamp_est'] = pd.DatetimeIndex(sportsbook_data['timestamp']).tz_localize(pytz.utc).tz_convert(eastern)
print(type(sportsbook_data['timestamp']))
print(type(sportsbook_data['timestamp_est']))
print(sportsbook_data.head())

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
                                      Date       Book Away Moneyline  \
0  Sun Sep 02 2018 15:37:47 GMT+0000 (UTC)   AlfaBook            115   
1  Thu Aug 30 2018 20:54:20 GMT+0000 (UTC)  BravoBook            NaN   
2  Thu Aug 30 2018 21:54:19 GMT+0000 (UTC)  BravoBook            NaN   
3  Thu Aug 30 2018 22:54:20 GMT+0000 (UTC)  BravoBook            NaN   
4  Thu Aug 30 2018 23:54:20 GMT+0000 (UTC)  BravoBook            NaN   

  Home Moneyline Away Pointspread points  Away Pointspread odds  \
0           -135                     2.5                 -110.0   
1            NaN                       3                 -119.0   
2            NaN                       3                 -110.0   
3            NaN                       3                 -110.0   
4            NaN                       3                 -110.0   

  Home Pointspread points  Home Pointspread odds  Total points  Over odds  \
0              

In [235]:
export_path = r'C:\Users\benny\OneDrive\Machine Learning\ML1030\Final Project\Final Project'
sportsbook_data.to_csv(export_path+"\\sportsbook_data.csv")

In [236]:
sportsbook_data_ALL =  pd.merge(sportsbook_data, nfl_game_schedule, on='Game', how='inner')

In [237]:
sportsbook_data_ALL['Pre-Game'] = ""
sportsbook_data_ALL['Pre-Game'] =  sportsbook_data_ALL['timestamp_est'][:] < sportsbook_data_ALL['Game Time'][:]
print(sportsbook_data_ALL.head())
export_path = r'C:\Users\benny\OneDrive\Machine Learning\ML1030\Final Project\Final Project'
sportsbook_data_ALL.to_csv(export_path+"\\sportsbook_data_ALL.csv")

                                    Date_x       Book Away Moneyline  \
0  Sun Sep 02 2018 15:37:47 GMT+0000 (UTC)   AlfaBook            115   
1  Thu Aug 30 2018 20:54:20 GMT+0000 (UTC)  BravoBook            NaN   
2  Thu Aug 30 2018 21:54:19 GMT+0000 (UTC)  BravoBook            NaN   
3  Thu Aug 30 2018 22:54:20 GMT+0000 (UTC)  BravoBook            NaN   
4  Thu Aug 30 2018 23:54:20 GMT+0000 (UTC)  BravoBook            NaN   

  Home Moneyline Away Pointspread points  Away Pointspread odds  \
0           -135                     2.5                 -110.0   
1            NaN                       3                 -119.0   
2            NaN                       3                 -110.0   
3            NaN                       3                 -110.0   
4            NaN                       3                 -110.0   

  Home Pointspread points  Home Pointspread odds  Total points  Over odds  \
0                    -2.5                 -110.0          45.5     -108.0   
1         